In [1]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *

In [2]:
from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals
from tensorly.decomposition import non_negative_parafac

def check_rank(tensor, rank, non_neg=True, n=10, tol=0.0001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                # print('doing', k)
                # print(full)
                print(tl.max(abs(diff)))
                print(factors)
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
                print(factors)
            if tl.max(abs(diff)) < tol:
                return True
    return False


In [3]:
#generates random rank 3 tensors
def rank_tree():
    return (low_tensor() + low_tensor() + low_tensor()) 

#generates random rank 4+ (hopefully) tensors
def rank_for():
    return  (low_tensor() + low_tensor() + low_tensor() + low_tensor())


# generates rank 1, 2x2x3 tensors
def low_tensor():
    max = 300
    a = np.random.randint(1, max, size=3) * 0.1
    b = np.random.randint(1, max, size=2)* 0.1
    c = np.random.randint(1, max, size=2)* 0.1
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2)) * 1.0
    return tens

def low_tensor_inv():
    max = 300
    a = np.random.randint(1, max, size=2) * -0.1
    b = np.random.randint(1, max, size=3)* 0.1
    c = np.random.randint(1, max, size=3)* 0.1
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(2, 3, 3)) * 1.0
    return tens

def check(t):
    t1 = tl.tensor([t[0], t[1]])
    t2 = tl.tensor([t[0], t[2]])
    a1 = det(Matrix(t1[0]))
    a2 = det(Matrix(t1[1]))
    a3 = det(Matrix(t2[0]))
    a4 = det(Matrix(t2[1]))
    b1 = det(Matrix(t1[:,0]))
    b2 = det(Matrix(t1[:,1]))
    b3 = det(Matrix(t2[:,0]))
    b4 = det(Matrix(t2[:,1]))
    c1 = det(Matrix(t1[:,:,0]))
    c2 = det(Matrix(t1[:,:,1]))
    c3 = det(Matrix(t2[:,:,0]))
    c4 = det(Matrix(t2[:,:,1]))
    return sgn(a1, a2,a3,a4) or sgn(b1,b2,b3,b4) or sgn(c1,c2,c3,c4)

def sgn(a1,a2,a3,a4):
    return abs(a1) + abs(a2) + abs(a3) + abs(a4) == abs(a1 + a2 + a3 + a4)
    

In [61]:
counter = 0
for i in range(10000):
    t = rank_for() + rank_for() + rank_for()  + rank_for() + rank_for() + rank_for() + rank_for() + rank_for()
    if check(t):
        #print(check_rank(t,3,n=20))
        counter = counter+1
print(counter)


8772


In [57]:
A = [1,0]
B = [0,1]
C1 = [1,0,0]
C2 = [0,1,0]
t1 = np.kron(np.kron(C1,A),A).reshape(3,2,2)
t2 = np.kron(np.kron(C1,B),B).reshape(3,2,2)
t3=np.kron(np.kron(C2,B),A).reshape(3,2,2)
t4=np.kron(np.kron(C2,A),B).reshape(3,2,2)
t = t1 + t2 + t3 + t4 + low_tensor()
print(t)
check_rank(t,3, p = True)

[[[ 8911.975  8800.05 ]
  [10324.44  10196.92 ]]

 [[ 8910.975  8801.05 ]
  [10325.44  10195.92 ]]

 [[ 7478.23   7385.14 ]
  [ 8664.432  8556.576]]]
6.255981006200784e-05
[array([[19559.74358895,   175.15549471,  9145.1734489 ],
       [19557.99983832,   176.0667306 ,  9146.64180264],
       [16413.07944061,   147.35350563,  7675.29995231]]), array([[0.70558096, 2.33323841, 0.52805009],
       [0.71602663, 0.        , 1.08204479]]), array([[0.55455427, 0.02391339, 0.25854093],
       [0.51090596, 0.64833218, 0.30732201]])]


True

In [217]:
B = P1.transpose() * P2.transpose()
B.eigenvects()

[(49.1666666666667,
  1,
  [Matrix([
   [ -0.917378869030917],
   [-0.0867790822056271],
   [ -0.388439701301379]])]),
 (1.05319148936148,
  1,
  [Matrix([
   [-0.828980717806854],
   [ -1.07995653145482],
   [-0.308015771295232]])]),
 (0.434944237918307,
  1,
  [Matrix([
   [-0.783472770444663],
   [  -1.0813219228451],
   [-0.369073949713589]])])]

In [182]:
C = P1 * P3
C.eigenvects()

[(14.6470588235294,
  1,
  [Matrix([
   [-0.379470952289146],
   [-0.917526780161815],
   [-0.118938656687643]])]),
 (0.931034482758621,
  1,
  [Matrix([
   [-0.143597906725126],
   [  -1.0105037880657],
   [-0.154234788704765]])]),
 (0.267857142857143,
  1,
  [Matrix([
   [-0.0974234014813838],
   [ -0.201805617354295],
   [ -0.285311390052624]])])]

In [210]:
t

array([[[16920.192 , 28519.892 , 20423.842 ],
        [ 6208.416 ,  8137.986 ,  5379.321 ],
        [ 2023.359 ,  4677.409 ,  2567.234 ]],

       [[15968.9336, 18838.7496, 14788.3776],
        [ 6340.1426,  6713.5946,  5204.4506],
        [ 1694.2368,  2401.3528,  1620.4508]]])